In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xlrd
import csv
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

def num_missing(x):
    return sum(x.isnull())

def missing(data):
    print(data.apply(num_missing,axis=0))
# Any results you write to the current directory are saved as output.

['kddcup99', 'kddcup99data', 'kdd-train-pre']


In [2]:
class config:
    kdd99train='../input/kddcup99data/kddcup.data'
#     kdd99test='../input/kddcup99/KDDCup99.csv'
    kdd99test='../input/kdd-train-pre/train_preprocess.csv'

def attack_class(train):
    train.loc[(train['label'].isin(['back.','land.','neptune.','pod.','smurf.','teardrop.'])),'label_type']='DOS'
    train.loc[train['label'].isin(['ipsweep.','nmap.','portsweep.','satan.']),'label_type']='Probing'
    train.loc[train['label'].isin(['ftp_write.','guess_passwd.','imap.','multihop.','phf.','spy.','warezclient.','warezmaster.']),'label_type']='R2L'
    train.loc[train['label'].isin(['buffer_overflow.','loadmodule.','perl.','rootkit.']),'label_type']='U2R'
    train.loc[train['label']=='normal.','label_type']='Normal'
    return train

def attack_class2(train):
    train.loc[(train['label'].isin(['back','land','neptune','pod','smurf','teardrop'])),'label_type']='DOS'
    train.loc[train['label'].isin(['ipsweep','nmap','portsweep','satan']),'label_type']='Probing'
    train.loc[train['label'].isin(['ftp_write','guess_passwd','imap','multihop','phf','spy','warezclient','warezmaster','dict']),'label_type']='R2L'
    train.loc[train['label'].isin(['buffer_overflow','loadmodule','perl','rootkit','ffb','eject','perlmagic']),'label_type']='U2R'
    train.loc[train['label']=='normal','label_type']='Normal'
    return train


cleanup_nums = {"protocol_type":     {"tcp": 1, "icmp": 2, "udp": 3},
                "service": {"vmnet": 1, "smtp": 2, "ntp_u":3, "shell":4, "kshell":5, "aol":6, "imap4":7, "urh_i":8, "netbios_ssn":9,
                           "tftp_u":10, "mtp":11, "uucp":12, "nnsp":13, "echo":14, "tim_i":15, "ssh":16, "iso_tsap":17, "time":18,
                           "netbios_ns":19,"systat":20, "hostnames":21, "login":22, "efs":23, "supdup":24, "http_8001":25, "courier":26,
                           "ctf":27,"finger":28,"nntp":29,"ftp_data":30,"red_i":31,"ldap":32,"http":33,"ftp":34,"pm_dump":35,"exec":36,
                           "klogin":37,"auth":38,"netbios_dgm":39,"other":40,"link":41,"X11":42,"discard":43,"private":44,"remote_job":45,
                           "IRC":46,"daytime":47,"pop_3":48,"pop_2":49,"gopher":50,"sunrpc":51,"name":52,"rje":53,"domain":54,"uucp_path":55,
                           "http_2784":56,"Z39_50":57,"domain_u":58,"csnet_ns":59,"whois":60,"eco_i":61,"bgp":62,"sql_net":63,"printer":64,
                           "telnet":65,"ecr_i":66,"urp_i":67,"netstat":68,"http_443":69,"harvest":70,"oth_i":71, "icmp":72},
               "flag":{"RSTR":1,"S3":2,"SF":3,"RSTO":4,"SH":5,"OTH":6,"S2":7,"RSTOS0":8,"S1":9,"S0":10,"REJ":11},
               "label_type":{"Normal":1,"DOS":2,"Probing":3,"R2L":4,"U2R":5}}
uni_cleanup_nums = {"protocol_type":     {"tcp": 1, "icmp": 2, "udp": 3},
                "service": {"vmnet": 1, "smtp": 2, "ntp_u":3, "shell":4, "kshell":5, "aol":6, "imap4":7, "urh_i":8, "netbios_ssn":9,
                           "tftp_u":10, "mtp":11, "uucp":12, "nnsp":13, "echo":14, "tim_i":15, "ssh":16, "iso_tsap":17, "time":18,
                           "netbios_ns":19,"systat":20, "hostnames":21, "login":22, "efs":23, "supdup":24, "http_8001":25, "courier":26,
                           "ctf":27,"finger":28,"nntp":29,"ftp_data":30,"red_i":31,"ldap":32,"http":33,"ftp":34,"pm_dump":35,"exec":36,
                           "klogin":37,"auth":38,"netbios_dgm":39,"other":40,"link":41,"X11":42,"discard":43,"private":44,"remote_job":45,
                           "IRC":46,"daytime":47,"pop_3":48,"pop_2":49,"gopher":50,"sunrpc":51,"name":52,"rje":53,"domain":54,"uucp_path":55,
                           "http_2784":56,"Z39_50":57,"domain_u":58,"csnet_ns":59,"whois":60,"eco_i":61,"bgp":62,"sql_net":63,"printer":64,
                           "telnet":65,"ecr_i":66,"urp_i":67,"netstat":68,"http_443":69,"harvest":70},
               "flag":{"RSTR":1,"S3":2,"SF":3,"RSTO":4,"SH":5,"OTH":6,"S2":7,"RSTOS0":8,"S1":9,"S0":10,"REJ":11},
               "label_type":{"Normal":1,"DOS":2,"Probing":3,"R2L":4,"U2R":5}}

attack_map={"label_type":{"Normal":1,"DOS":2,"Probing":3,"R2L":4,"U2R":5}}



column = ["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins","logged_in","lnum_compromised","lroot_shell","lsu_attempted","lnum_root","lnum_file_creations","lnum_shells","lnum_access_files","lnum_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

save_features = ["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]


In [3]:
train = pd.read_csv(config.kdd99train)
train.columns= column
train['service'].value_counts()

ecr_i          2811660
private        1100831
http            623090
smtp             96554
other            72653
domain_u         57782
ftp_data         40697
eco_i            16338
finger            6891
urp_i             5378
ftp               5214
telnet            4277
ntp_u             3833
auth              3382
pop_3             1981
time              1579
domain            1113
Z39_50            1078
gopher            1077
mtp               1076
ssh               1075
whois             1073
remote_job        1073
rje               1070
link              1069
imap4             1069
ctf               1068
name              1067
supdup            1060
nntp              1059
                ...   
vmnet             1053
iso_tsap          1052
netbios_dgm       1052
sql_net           1052
shell             1051
csnet_ns          1051
klogin            1050
hostnames         1050
bgp               1047
login             1045
printer           1045
exec              1045
http_443   

In [4]:
%%time
### 加载数据，特征映射，类别分类

###读取训练集
print("train data loading")
train = pd.read_csv(config.kdd99train)
# 设置属性
train.columns= column
# 选取相同的属性
train = train[save_features]

train.insert(0,'label_type','NAN')
# train.info()
# train.head()
# train['label'].value_counts()
# train['label_type'].value_counts()
#Mapping of Categorical data in the dataset


###读取测试机
print("test data loading")
test = pd.read_csv(config.kdd99test)

# 选取相同的属性
test = test[save_features]

test.insert(0,'label_type','NAN')
# test.info()
# test.head()
# test['label'].value_counts()



train data loading
test data loading
CPU times: user 18.5 s, sys: 2.22 s, total: 20.7 s
Wall time: 20.7 s


In [5]:
def drop(datapre):
    #Dropping all the other unselecteced attributes from dataset
#     datapre.drop('duration', axis = 1, inplace = True)
#     print("Dropping duration")
    # datapre.drop('flag', axis = 1, inplace = True)
    # print("Dropping flag")
#     datapre.drop('dst_bytes', axis = 1, inplace = True)
#     print("Dropping dst_bytes")
#     datapre.drop('land', axis = 1, inplace = True)
#     print("Dropping land")
    datapre.drop('wrong_fragment', axis = 1, inplace = True)
    print("Dropping wrong_fragment")
    datapre.drop('urgent', axis = 1, inplace = True)
    print("Dropping urgent")
#     datapre.drop('hot', axis = 1, inplace = True)
#     print("Dropping hot")
#     datapre.drop('num_failed_logins', axis = 1, inplace = True)
#     print("Dropping num_failed_logins")
#     datapre.drop('logged_in', axis = 1, inplace = True)
#     print("Dropping logged_in")
#     datapre.drop('lnum_compromised', axis = 1, inplace = True)
#     print("Dropping lnum_compromised")
#     datapre.drop('lroot_shell', axis = 1, inplace = True)
#     print("Dropping lroot_shell")
#     datapre.drop('lsu_attempted', axis = 1, inplace = True)
#     print("Dropping lsu_attempted")
#     datapre.drop('lnum_root', axis = 1, inplace = True)
#     print("Dropping lnum_root")
#     datapre.drop('lnum_file_creations', axis = 1, inplace = True)
#     print("Dropping lnum_file_creations")
#     datapre.drop('lnum_shells', axis = 1, inplace = True)
#     print("Dropping lnum_shells")
#     datapre.drop('lnum_access_files', axis = 1, inplace = True)
#     print("Dropping lnum_access_files")
#     datapre.drop('lnum_outbound_cmds', axis = 1, inplace = True)
#     print("Dropping lnum_outbound_cmds")
#     datapre.drop('is_host_login', axis = 1, inplace = True)
#     print("Dropping is_host_login")
#     datapre.drop('is_guest_login', axis = 1, inplace = True)
#     print("Dropping is_guest_login")
    # datapre.drop('srv_count', axis = 1, inplace = True)
    # print("Dropping srv_count")
    # datapre.drop('serror_rate', axis = 1, inplace = True)
    # print("Dropping serror_rate")
    # datapre.drop('srv_serror_rate', axis = 1, inplace = True)
    # print("Dropping srv_serror_rate")
    datapre.drop('rerror_rate', axis = 1, inplace = True)
    print("Dropping rerror_rate")
    datapre.drop('srv_rerror_rate', axis = 1, inplace = True)
    print("Dropping srv_rerror_rate")
#     datapre.drop('diff_srv_rate', axis = 1, inplace = True)
#     print("Dropping diff_srv_rate")
#     datapre.drop('srv_diff_host_rate', axis = 1, inplace = True)
#     print("Dropping srv_diff_host_rate")
    # datapre.drop('dst_host_count', axis = 1, inplace = True)
    # print("dst_host_count")
#     datapre.drop('dst_host_srv_count', axis = 1, inplace = True)
#     print("Dropping dst_host_srv_count")
    # datapre.drop('dst_host_diff_srv_rate', axis = 1, inplace = True)
    # print("Dropping dst_host_diff_srv_rate")
#     datapre.drop('dst_host_srv_diff_host_rate', axis = 1, inplace = True)
#     print("Dropping dst_host_srv_diff_host_rate")
#     datapre.drop('dst_host_serror_rate', axis = 1, inplace = True)
#     print("Dropping dst_host_serror_rate")
#     datapre.drop('dst_host_srv_serror_rate', axis = 1, inplace = True)
#     print("Dropping dst_host_srv_serror_rate")
    datapre.drop('dst_host_rerror_rate', axis = 1, inplace = True)
    print("Dropping dst_host_rerror_rate")
    datapre.drop('dst_host_srv_rerror_rate', axis = 1, inplace = True)
    print("Dropping dst_host_srv_rerror_rate")
    return datapre
# train = drop(train)
test = drop(test)

Dropping wrong_fragment
Dropping urgent
Dropping rerror_rate
Dropping srv_rerror_rate
Dropping dst_host_rerror_rate
Dropping dst_host_srv_rerror_rate


In [6]:
list(set(train['flag']).difference(set(test['flag'])))

['RSTOS0', 'REJ', 'S0', 'SH']

In [7]:
list(set(test['flag']).difference(set(train['flag'])))

[]

In [8]:
list(set(train['service']).difference(set(test['service'])))

['harvest', 'private', 'red_i', 'urp_i', 'urh_i', 'pm_dump']

In [9]:
list(set(test['service']).difference(set(train['service'])))

['icmp', 'oth_i']

In [10]:
list(set(train['label']).difference(set(test['label'])))

['neptune.',
 'buffer_overflow.',
 'nmap.',
 'warezclient.',
 'normal.',
 'satan.',
 'back.',
 'guess_passwd.',
 'phf.',
 'pod.',
 'loadmodule.',
 'multihop.',
 'smurf.',
 'teardrop.',
 'rootkit.',
 'perl.',
 'ipsweep.',
 'warezmaster.',
 'spy.',
 'portsweep.',
 'imap.',
 'land.',
 'ftp_write.']

In [11]:
list(set(test['label']).difference(set(train['label'])))

['land',
 'neptune',
 'satan',
 'teardrop',
 'normal',
 'ffb',
 'phf',
 'portsweep',
 'dict',
 'eject',
 'smurf',
 'ipsweep',
 'perlmagic',
 'pod']

In [12]:

datapre = train
# datapre.transpose()
#Replacing the encoded data in the dataset
datapre.replace(cleanup_nums, inplace=True)
# datapre.transpose()
# datapre.head()


train = attack_class(datapre)

train['label_type'].value_counts()

# df[['two', 'three']] = df[['two', 'three']].astype(float)
train.replace(attack_map, inplace=True)
# train.head()
# train.drop('label', axis = 1, inplace = True)
# train.head()


datapre = test
# datapre.transpose()
#Replacing the encoded data in the dataset
datapre.replace(cleanup_nums, inplace=True)
# datapre.transpose()
# datapre.head()
# datapre.head()
test = attack_class2(datapre)

test['label_type'].value_counts()

# df[['two', 'three']] = df[['two', 'three']].astype(float)
test.replace(attack_map, inplace=True)
# test.head()
# test.drop('label', axis = 1, inplace = True)
test.head()



,label_type,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,count,srv_count,serror_rate,srv_serror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,label
0,1,0,2,66,3,64,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,normal
1,1,0,2,71,3,1296,0,0,1,0,0.0,0.0,0.0,1.0,0.0,1,0,0.0,1.0,0.0,0.0,0.0,0.0,normal
2,1,0,1,2,3,1409,957,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,normal
3,1,5,1,2,3,1413,940,0,0,1,0.0,0.0,0.0,0.0,1.0,0,1,0.0,0.0,0.0,1.0,0.0,0.0,normal
4,1,4,1,2,3,1354,941,0,0,2,0.0,0.0,0.0,0.0,1.0,0,2,0.0,0.0,0.0,1.0,0.0,0.0,normal


In [13]:
# set(test.loc[test['label_type']=='NAN']['label'])

In [14]:
# def drop(datapre):
#     #Dropping all the other unselecteced attributes from dataset
#     datapre.drop('duration', axis = 1, inplace = True)
#     print("Dropping duration")
#     # datapre.drop('flag', axis = 1, inplace = True)
#     # print("Dropping flag")
#     datapre.drop('dst_bytes', axis = 1, inplace = True)
#     print("Dropping dst_bytes")
#     datapre.drop('land', axis = 1, inplace = True)
#     print("Dropping land")
#     datapre.drop('wrong_fragment', axis = 1, inplace = True)
#     print("Dropping wrong_fragment")
#     datapre.drop('urgent', axis = 1, inplace = True)
#     print("Dropping urgent")
#     datapre.drop('hot', axis = 1, inplace = True)
#     print("Dropping hot")
#     datapre.drop('num_failed_logins', axis = 1, inplace = True)
#     print("Dropping num_failed_logins")
#     datapre.drop('logged_in', axis = 1, inplace = True)
#     print("Dropping logged_in")
#     datapre.drop('lnum_compromised', axis = 1, inplace = True)
#     print("Dropping lnum_compromised")
#     datapre.drop('lroot_shell', axis = 1, inplace = True)
#     print("Dropping lroot_shell")
#     datapre.drop('lsu_attempted', axis = 1, inplace = True)
#     print("Dropping lsu_attempted")
#     datapre.drop('lnum_root', axis = 1, inplace = True)
#     print("Dropping lnum_root")
#     datapre.drop('lnum_file_creations', axis = 1, inplace = True)
#     print("Dropping lnum_file_creations")
#     datapre.drop('lnum_shells', axis = 1, inplace = True)
#     print("Dropping lnum_shells")
#     datapre.drop('lnum_access_files', axis = 1, inplace = True)
#     print("Dropping lnum_access_files")
#     datapre.drop('lnum_outbound_cmds', axis = 1, inplace = True)
#     print("Dropping lnum_outbound_cmds")
#     datapre.drop('is_host_login', axis = 1, inplace = True)
#     print("Dropping is_host_login")
#     datapre.drop('is_guest_login', axis = 1, inplace = True)
#     print("Dropping is_guest_login")
#     # datapre.drop('srv_count', axis = 1, inplace = True)
#     # print("Dropping srv_count")
#     # datapre.drop('serror_rate', axis = 1, inplace = True)
#     # print("Dropping serror_rate")
#     # datapre.drop('srv_serror_rate', axis = 1, inplace = True)
#     # print("Dropping srv_serror_rate")
#     # datapre.drop('rerror_rate', axis = 1, inplace = True)
#     # print("Dropping rerror_rate")
#     # datapre.drop('srv_rerror_rate', axis = 1, inplace = True)
#     # print("Dropping srv_rerror_rate")
#     datapre.drop('diff_srv_rate', axis = 1, inplace = True)
#     print("Dropping diff_srv_rate")
#     datapre.drop('srv_diff_host_rate', axis = 1, inplace = True)
#     print("Dropping srv_diff_host_rate")
#     # datapre.drop('dst_host_count', axis = 1, inplace = True)
#     # print("dst_host_count")
#     datapre.drop('dst_host_srv_count', axis = 1, inplace = True)
#     print("Dropping dst_host_srv_count")
#     # datapre.drop('dst_host_diff_srv_rate', axis = 1, inplace = True)
#     # print("Dropping dst_host_diff_srv_rate")
#     datapre.drop('dst_host_srv_diff_host_rate', axis = 1, inplace = True)
#     print("Dropping dst_host_srv_diff_host_rate")
#     # datapre.drop('dst_host_serror_rate', axis = 1, inplace = True)
#     # print("Dropping dst_host_serror_rate")
#     # datapre.drop('dst_host_srv_serror_rate', axis = 1, inplace = True)
#     # print("Dropping dst_host_srv_serror_rate")
#     # datapre.drop('dst_host_rerror_rate', axis = 1, inplace = True)
#     # print("Dropping dst_host_rerror_rate")
#     # datapre.drop('dst_host_srv_rerror_rate', axis = 1, inplace = True)
#     # print("Dropping dst_host_srv_rerror_rate")
#     return datapre
# train = drop(train)
# test = drop(test)

In [15]:
train.shape

(4898430, 30)

In [16]:
test.shape

(534584, 24)

In [17]:
test_data=test.values
X_input=test_data[:,1:23]
y_input=test_data[:,0]
#splitting data
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
features_train, features_test, labels_train, labels_test = train_test_split(X_input, y_input, test_size = 0.25)
labels_test=labels_test.astype(float)
labels_train=labels_train.astype(float)

In [18]:
set(labels_train)

{1.0, 2.0, 3.0, 4.0, 5.0}

In [25]:
%%time
#Naive Bayes Classifier
from sklearn import metrics
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
clf = GaussianNB()
clf.fit(features_train, labels_train)
prediction = clf.predict(features_test)
acc=accuracy_score(prediction, labels_test)
print("------------------------------------------")
print("Accuracy = ",acc*100," %")
matrix = confusion_matrix(labels_test, prediction)
print(matrix)
report = classification_report(labels_test, prediction)
print(report)

# from sklearn.metrics import roc_curve
# from sklearn.metrics import auc
# from sklearn import metrics
# fpr, tpr, threshold = metrics.roc_curve(labels_test, prediction)
# roc_auc = metrics.auc(fpr, tpr)
# import matplotlib.pyplot as plt
# plt.title('Receiver Operating Characteristic')
# plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
# plt.legend(loc = 'lower right')
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0, 1])
# plt.ylim([0, 1])
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.show()

ValueError: Unknown label type: (array([1, 2, 3, 4, 5], dtype=object),)

In [20]:
%%time
#Random Forest Classifier
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(features_train, labels_train)
prediction = clf.predict(features_test)
acc=accuracy_score(prediction, labels_test)
print("------------------------------------------")
print("Accuracy = ",acc*100," %")
matrix = confusion_matrix(labels_test, prediction)
print(matrix)

report = classification_report(labels_test, prediction)
print(report)


# from sklearn.metrics import roc_curve
# from sklearn.metrics import auc
# from sklearn import metrics
# fpr, tpr, threshold = metrics.roc_curve(labels_test, prediction)
# roc_auc = metrics.auc(fpr, tpr)
# import matplotlib.pyplot as plt
# plt.title('Receiver Operating Characteristic')
# plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
# plt.legend(loc = 'lower right')
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0, 1])
# plt.ylim([0, 1])
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.show()

# models.append("Random Forest Classifier")
# scores.append(acc*100)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


------------------------------------------
Accuracy =  99.46126333747362  %
[[ 18085    196    168      1      0]
 [   300 113455      0      0      0]
 [    48      0   1162      0      0]
 [     3      0      0    224      0]
 [     4      0      0      0      0]]
              precision    recall  f1-score   support

         1.0       0.98      0.98      0.98     18450
         2.0       1.00      1.00      1.00    113755
         3.0       0.87      0.96      0.91      1210
         4.0       1.00      0.99      0.99       227
         5.0       0.00      0.00      0.00         4

   micro avg       0.99      0.99      0.99    133646
   macro avg       0.77      0.78      0.78    133646
weighted avg       0.99      0.99      0.99    133646

CPU times: user 4.9 s, sys: 4 ms, total: 4.9 s
Wall time: 4.91 s


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
#K-Nearest Neighbour Classifier
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
# fit a k-nearest neighbor model to the data
clf = KNeighborsClassifier()
clf.fit(features_train, labels_train)
prediction = clf.predict(features_test)
acc=accuracy_score(prediction, labels_test)
print("------------------------------------------")
print("Accuracy = ",acc*100," %")
matrix = confusion_matrix(labels_test, prediction)
print(matrix)

report = classification_report(labels_test, prediction)
print(report)


# from sklearn.metrics import roc_curve
# from sklearn.metrics import auc
# from sklearn import metrics
# fpr, tpr, threshold = metrics.roc_curve(labels_test, prediction)
# roc_auc = metrics.auc(fpr, tpr)
# import matplotlib.pyplot as plt
# plt.title('Receiver Operating Characteristic')
# plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
# plt.legend(loc = 'lower right')
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0, 1])
# plt.ylim([0, 1])
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.show()

# models.append("K-Nearest neighbour Classifier")
# scores.append(acc*100)

------------------------------------------
Accuracy =  99.78375708962483  %
[[ 18271     21    151      7      0]
 [    13 113742      0      0      0]
 [    93      0   1117      0      0]
 [     0      0      0    227      0]
 [     4      0      0      0      0]]
              precision    recall  f1-score   support

         1.0       0.99      0.99      0.99     18450
         2.0       1.00      1.00      1.00    113755
         3.0       0.88      0.92      0.90      1210
         4.0       0.97      1.00      0.98       227
         5.0       0.00      0.00      0.00         4

   micro avg       1.00      1.00      1.00    133646
   macro avg       0.77      0.78      0.78    133646
weighted avg       1.00      1.00      1.00    133646



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
#Decision Tree Classisfier 
from sklearn import tree
clf = tree.DecisionTreeClassifier(splitter = "random", criterion = "entropy", max_depth = 40)
 
clf.fit(features_train, labels_train)
prediction = clf.predict(features_test)
## Computing accuracy
from sklearn.metrics import accuracy_score
acc=accuracy_score(prediction, labels_test)
print("------------------------------------------")
print("Accuracy = ",acc*100," %")
matrix = confusion_matrix(labels_test, prediction)
print(matrix)
report = classification_report(labels_test, prediction)
print(report)


# from sklearn.metrics import roc_curve
# from sklearn.metrics import auc
# from sklearn import metrics
# fpr, tpr, threshold = metrics.roc_curve(labels_test, prediction)
# roc_auc = metrics.auc(fpr, tpr)
# import matplotlib.pyplot as plt
# plt.title('Receiver Operating Characteristic')
# plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
# plt.legend(loc = 'lower right')
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0, 1])
# plt.ylim([0, 1])
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.show()

# models.append("Decision tree Classifier")
# scores.append(acc*100)

------------------------------------------
Accuracy =  99.76280621941547  %
[[ 18241     33    175      1      0]
 [    48 113707      0      0      0]
 [    56      0   1154      0      0]
 [     0      0      0    227      0]
 [     3      0      0      1      0]]
              precision    recall  f1-score   support

         1.0       0.99      0.99      0.99     18450
         2.0       1.00      1.00      1.00    113755
         3.0       0.87      0.95      0.91      1210
         4.0       0.99      1.00      1.00       227
         5.0       0.00      0.00      0.00         4

   micro avg       1.00      1.00      1.00    133646
   macro avg       0.77      0.79      0.78    133646
weighted avg       1.00      1.00      1.00    133646



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [23]:
%%time
#Cross validation
# from sklearn import cross_validation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



sam1 = train.sample(frac=0.5,random_state=1)
sam = sam1.sample(frac=0.5,random_state=1)
train_data = sam.values
features_train = train_data[:,1:29]
labels_train = train_data[:,0]

test_data = test.values
features_test = test_data[:,1:29]
labels_test = test_data[:,0]

CPU times: user 4.25 s, sys: 1.03 s, total: 5.28 s
Wall time: 5.27 s


In [24]:
### load module
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from xgboost import plot_importance

### load datasets
# digits = datasets.load_digits()

### data analysis
# print(digits.data.shape)
# print(digits.target.shape)

### data split
# x_train,x_test,y_train,y_test = train_test_split(digits.data,
#                                                  digits.target,
#                                                  test_size = 0.3,
#                                                  random_state = 33)
x_train=features_train
y_train=labels_train
x_test=features_test
y_test=labels_test.astype(float)

### fit model for train data
model = XGBClassifier(learning_rate=0.1,
                      n_estimators=1000,         # 树的个数--1000棵树建立xgboost
                      max_depth=10,               # 树的深度
                      min_child_weight = 1,      # 叶子节点最小权重
                      gamma=0.,                  # 惩罚项中叶子结点个数前的参数
                      subsample=0.8,             # 随机选择80%样本建立决策树
                      colsample_btree=0.8,       # 随机选择80%特征建立决策树
                      objective='multi:softmax', # 指定损失函数
                      scale_pos_weight=1,        # 解决样本个数不平衡的问题
                      random_state=27            # 随机数
                      )
model.fit(x_train,
          y_train,
          eval_set = [(x_test,y_test)],
          eval_metric = "mlogloss",
          early_stopping_rounds = 10,
          verbose = True)

### plot feature importance
fig,ax = plt.subplots(figsize=(15,15))
plot_importance(model,
                height=0.5,
                ax=ax,
                max_num_features=64)
plt.show()

### make prediction for test data
y_pred = model.predict(x_test)


feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

### model evaluate
accuracy = accuracy_score(y_test,y_pred)
print("accuarcy: %.2f%%" % (accuracy*100.0))


ValueError: could not convert string to float: 'normal'

In [ ]:
matrix = confusion_matrix(y_test.astype(float),y_pred)
print(matrix)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
model1 = XGBClassifier(learning_rate=0.1,
                      n_estimators=1000,         # 树的个数--1000棵树建立xgboost
                      max_depth=10,               # 树的深度
                      min_child_weight = 1,      # 叶子节点最小权重
                      gamma=0.,                  # 惩罚项中叶子结点个数前的参数
                      subsample=0.8,             # 随机选择80%样本建立决策树
                      colsample_btree=0.8,       # 随机选择80%特征建立决策树
                      objective='multi:softmax', # 指定损失函数
                      scale_pos_weight=1,        # 解决样本个数不平衡的问题
                      )
learning_rate = [0.005, 0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learning_rate=learning_rate)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model1, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_input, y_input)

In [ ]:
feat_imp = pd.Series(model.booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

In [ ]:
set(y_test)

In [ ]:
set(y_pred)


In [ ]:
set(y_pred.astype(int))
y_p=y_pred.astype(int)

In [ ]:
set(y_test.astype(float))

In [ ]:

### model evaluate
accuracy = accuracy_score(y_test.astype(float),y_pred)
print("accuarcy: %.2f%%" % (accuracy*100.0))

In [ ]:
num=0
for i in range(y_test.shape[0]):
    if(y_test[i]==y_p[i]):
        num=num+1
        
print(num*1.0/(y_test.shape[0]))

num=0
for i in range(y_test.shape[0]):
    if(y_test[i]==y_p[i]):
        num=num+1
    elif ((y_test[i]!=1) & (y_p[i]!=1)):
        num=num+1
        
print(num*1.0/(y_test.shape[0]))

In [ ]:
### model evaluate
accuracy = accuracy_score(y_test,y_pred.astype(int))
print("accuarcy: %.2f%%" % (accuracy*100.0))

In [ ]:
train.info()

In [ ]:
test.info()


In [ ]:
# #Cross validation
# # from sklearn import cross_validation
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report

# train_data = train.values
# features_train = train_data[:,1:42]
# labels_train = train_data[:,0]

# test_data = test.values
# features_test = test_data[:,1:42]
# labels_test = test_data[:,0]

In [ ]:
# #Naive Bayes Classifier
# from sklearn import metrics
# from sklearn import tree
# from sklearn.naive_bayes import GaussianNB
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import mean_squared_error
# clf = GaussianNB()
# clf.fit(features_train, labels_train)
# prediction = clf.predict(features_test)
# acc=accuracy_score(prediction, labels_test)
# print("------------------------------------------")
# print("Accuracy = ",acc*100," %")
# matrix = confusion_matrix(labels_test, prediction)
# print(matrix)
# report = classification_report(labels_test, prediction)
# print(report)

# from sklearn.metrics import roc_curve
# from sklearn.metrics import auc
# from sklearn import metrics
# fpr, tpr, threshold = metrics.roc_curve(labels_test, prediction)
# roc_auc = metrics.auc(fpr, tpr)
# import matplotlib.pyplot as plt
# plt.title('Receiver Operating Characteristic')
# plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
# plt.legend(loc = 'lower right')
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0, 1])
# plt.ylim([0, 1])
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.show()

In [ ]:
# #Applying Logistic regression to find important features. Higher the rank more important the attribute
# from sklearn.feature_selection import RFE
# from sklearn.linear_model import LogisticRegression
# model = LogisticRegression()
# rfe = RFE(model, 10)
# fit = rfe.fit(features_train, labels_train)
# print("Num Features: ",fit.n_features_)
# print("Selected Features: ",fit.support_)
# print("Ranking of features: ",fit.ranking_)